## Modules

In [52]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
from datetime import datetime, timedelta
from itertools import product

import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from scipy.stats import skew, kurtosis, linregress, norm
import yfinance as yf

ohlcfield = ['op', 'hi', 'lo', 'cl']   
ohlcvdfield = ['op', 'hi', 'lo', 'cl', 'vol', 'div']

## Log-linear Regression channel

### Yahoo OHLC data

In [53]:
def getyahoodata(symbollist, adjust=True, startstr='1990-01-01', endstr='2046-12-31'):
    """Scrape via yahoo API to obtain data for a symbollist."""
    symbolstr = ' '.join(symbollist)
    renamedict = {'Date': 'date', 'Open': 'op', 'High': 'hi', 'Low': 'lo', 'Close': 'cl', 'Volume': 'vol',
                   'Adj Close': 'adj_cl', 'Dividends': 'div', 'Stock Splits': 'split'}
    ohlcvdfield = ['op', 'hi', 'lo', 'cl', 'vol', 'div']
    datadict = dict()

    try:
        dfdata = yf.download(symbolstr, start=startstr, end=endstr, auto_adjust=False, actions=True,
                                 group_by='Tickers', threads=16)
    except:
        dfdata = pd.DataFrame()

    for symbol in symbollist:
        try:
            dfsymbol = dfdata[(symbol, )].dropna()  # Raw data for the symbol
            dfsymbol = dfsymbol[(dfsymbol['Volume'] > 0) | (dfsymbol['High'] > dfsymbol['Low'])] # Filter bad data
            dfsymbol = dfsymbol.reset_index()
            dfsymbol = dfsymbol.rename(columns=renamedict)
            dfsymbol = dfsymbol.set_index('date')
            if not adjust:
                dfsymbol = dfsymbol[ohlcvdfield]
                dfsymbol = dfsymbol.rename(columns={field: f'{symbol}_{field}' for field in ohlcvdfield})
            else:
                adjfactor = dfsymbol['adj_cl'] / dfsymbol['cl']
                for field in ohlcvdfield[:-2]:
                    dfsymbol[f'adj_{field}'] = dfsymbol[field] * adjfactor
                dfsymbol['adj_vol'] = dfsymbol['vol'] / adjfactor
                dfsymbol = dfsymbol[[f'adj_{field}' for field in ohlcvdfield[:-1]]]
                dfsymbol = dfsymbol.rename(columns={f'adj_{field}': f'{symbol}_{field}' for field in ohlcvdfield[:-1]})
                dfsymbol = np.round(dfsymbol, 4)
            datadict[symbol] = dfsymbol
        except:
            print(f'Failed preparing data for {symbol}.')

    dfallsymbols = pd.concat(datadict.values(), axis=1)
    dfallsymbols = dfallsymbols.fillna(method='ffill')
    dfallsymbols = np.round(dfallsymbols, 4)

    return dfallsymbols

In [54]:
assetlist = ['0050.TW', '0056.TW', '2330.TW', '2454.TW', '2882.TW', '2002.TW', '3008.TW', '2881.TW']
dfall = getyahoodata(assetlist, True)
dfall.info()

[*********************100%***********************]  8 of 8 completed
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5495 entries, 2000-01-04 to 2022-05-27
Data columns (total 40 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   0050.TW_op   3278 non-null   float64
 1   0050.TW_hi   3278 non-null   float64
 2   0050.TW_lo   3278 non-null   float64
 3   0050.TW_cl   3278 non-null   float64
 4   0050.TW_vol  3278 non-null   float64
 5   0056.TW_op   3278 non-null   float64
 6   0056.TW_hi   3278 non-null   float64
 7   0056.TW_lo   3278 non-null   float64
 8   0056.TW_cl   3278 non-null   float64
 9   0056.TW_vol  3278 non-null   float64
 10  2330.TW_op   5495 non-null   float64
 11  2330.TW_hi   5495 non-null   float64
 12  2330.TW_lo   5495 non-null   float64
 13  2330.TW_cl   5495 non-null   float64
 14  2330.TW_vol  5495 non-null   float64
 15  2454.TW_op   5121 non-null   float64
 16  2454.TW_hi   5121 non-null   float64
 17  245

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshell.py:2960: PerformanceWarning:

indexing past lexsort depth may impact performance.

C:\Kit\Python\Envs\TMBA\lib\site-packages\IPython\core\interactiveshel

### Resample into different timeframe

In [55]:
def resampleohlc(asset, dfohlc, freq='W', startstr='1990-01-01', endstr='2046-12-31'):
    """"""
    aggrule = {f'{asset}_op': 'first', f'{asset}_hi': 'max', f'{asset}_lo': 'min', f'{asset}_cl': 'last'}
    dfasset = dfohlc.loc[startstr:endstr, [f'{asset}_{field}' for field in ohlcfield]]
    dfnew = dfasset.resample(freq, closed='right').agg(aggrule)
        
    return dfnew

In [56]:
startstr0 = '2015-01-01'
endstr0 = '2022-12-31'

newdict = {asset: resampleohlc(asset, dfall, 'W', startstr0, endstr0) for asset in assetlist}
newdict['0050.TW'].head(10)

,0050.TW_op,0050.TW_hi,0050.TW_lo,0050.TW_cl
date,,,,
2015-01-11,66.40,66.95,64.70,66.15
2015-01-18,65.80,66.40,65.35,65.50
2015-01-25,66.30,68.70,66.05,68.70
2015-02-01,68.55,69.25,68.00,68.00
2015-02-08,68.20,69.50,67.90,68.75
2015-02-15,68.75,69.65,68.20,69.45
2015-02-22,NaN,NaN,NaN,NaN
2015-03-01,69.95,70.90,69.95,70.30
2015-03-08,70.30,70.55,69.30,69.80


In [57]:
dfall.loc['2015-12':'2022-01', ['0050.TW_op', '0050.TW_cl']]

,0050.TW_op,0050.TW_cl
date,,
2015-12-01,61.70,62.40
2015-12-02,62.45,62.30
2015-12-03,61.90,62.40
2015-12-04,61.95,61.75
2015-12-07,62.20,62.50
...,...,...
2022-01-20,148.50,149.10
2022-01-21,143.90,142.65
2022-01-24,142.10,144.00


### Semi-log OHLC dataframe

In [58]:
def semilogohlc(asset, dfohlc, startstr, endstr):
    """Obtain semi-log OHLC dataframe, x-axis being year count & y-axis in log scale."""
    ohlcfield = ['op', 'hi', 'lo', 'cl']   
    dflog = dfohlc.loc[startstr:endstr, [f'{asset}_{field}' for field in ohlcfield]]
    dflog['yearcount'] = [(date - dflog.index[0]).days/364 for date in dflog.index]
    for field in ohlcfield:
        dflog[f'{asset}_log{field}'] = np.log(dflog[f'{asset}_{field}'])
    
    return dflog

In [59]:
logdict = {asset: semilogohlc(asset, df, startstr0, endstr0) for asset, df in newdict.items()}
logdict['0050.TW'].tail(10)

,0050.TW_op,0050.TW_hi,0050.TW_lo,0050.TW_cl,yearcount,0050.TW_logop,0050.TW_loghi,0050.TW_loglo,0050.TW_logcl
date,,,,,,,,,
2022-03-27,136.80,138.50,136.30,138.20,7.230769,4.918520,4.930870,4.914858,4.928702
2022-04-03,137.00,138.70,135.55,136.95,7.250000,4.919981,4.932313,4.909341,4.919616
2022-04-10,135.25,136.35,133.75,134.55,7.269231,4.907125,4.915225,4.895972,4.901936
2022-04-17,134.15,135.75,131.60,132.25,7.288462,4.898959,4.910815,4.879767,4.884694
2022-04-24,132.00,133.60,130.85,131.60,7.307692,4.882802,4.894850,4.874052,4.879767
2022-05-01,130.00,130.05,125.95,128.75,7.326923,4.867534,4.867919,4.835885,4.857873
2022-05-08,128.55,129.80,125.95,126.65,7.346154,4.856318,4.865995,4.835885,4.841427
2022-05-15,125.50,125.55,121.20,122.35,7.365385,4.832306,4.832704,4.797442,4.806886
2022-05-22,123.45,126.00,122.50,124.65,7.384615,4.815836,4.836282,4.808111,4.825510


### Timeline regression

In [60]:
def timelinereg(asset, dfohlc, freq='W', startstr='1990-01-01', endstr='2022-12-31', regfield='cl'):
    """Obtain timeline regression equation coefficients and different semi-log projection bands."""
    # Resample OHLC dataframe
    dfnew = resampleohlc(asset, dfohlc, freq, startstr, endstr)
    # Semilog OHLC
    dflog = semilogohlc(asset, dfnew, startstr, endstr)
    # Log-linear Regression Equation
    result = linregress(dflog['yearcount'], dflog[f'{asset}_log{regfield}'])
    slope = result.slope
    yint = result.intercept
    corr = result.rvalue
    # Regression mean dataframe
    dfband = dflog.copy()
    dfband[f'{asset}_logavg'] = yint + slope * dfband['yearcount']
    dfband[f'{asset}_avg'] = np.exp(yint) * np.exp(slope * dfband['yearcount'])
    # SD of prediction error
    stderror = (dflog[f'{asset}_log{regfield}'] - dfband[f'{asset}_logavg']).std()
    # Regression error bands
    dfband[f'{asset}_+2SD'] = dfband[f'{asset}_avg'] * np.exp(2.0 * stderror)
    dfband[f'{asset}_+1SD'] = dfband[f'{asset}_avg'] * np.exp(1.0 * stderror)
    dfband[f'{asset}_-1SD'] = dfband[f'{asset}_avg'] * np.exp(-1.0 * stderror)
    dfband[f'{asset}_-2SD'] = dfband[f'{asset}_avg'] * np.exp(-2.0 * stderror)
    dfband[f'{asset}_zscore'] = np.round((dflog[f'{asset}_log{regfield}'] - dfband[f'{asset}_logavg']) / stderror, 4)
    # Timeline regression dict
    regdict = {'asset': asset, 'startdate': startstr, 'enddate': endstr,
               'Log-linear equation': f'y = {np.round(slope, 6)} * x + {np.round(yint, 6)}',
               'Timeline equation': f'P = {np.exp(np.round(yint, 6))} * exp({np.round(slope, 6)} * x)',
               'R^2': corr**2, 'stderr': stderror} 
    

    return dfband, regdict

### Regression coefficients & augmented dataframe

In [61]:
dfband0, regdict0 = timelinereg(assetlist[0], dfall, 'M', startstr0, endstr0)
regdict0

{'asset': '0050.TW',
 'startdate': '2015-01-01',
 'enddate': '2022-12-31',
 'Log-linear equation': 'y = 0.108946 * x + 4.063292',
 'Timeline equation': 'P = 58.16547699749018 * exp(0.108946 * x)',
 'R^2': 0.8205704451394082,
 'stderr': 0.11006975469966615}

In [62]:
dfband0.tail(10)

,0050.TW_op,0050.TW_hi,0050.TW_lo,0050.TW_cl,yearcount,0050.TW_logop,0050.TW_loghi,0050.TW_loglo,0050.TW_logcl,0050.TW_logavg,0050.TW_avg,0050.TW_+2SD,0050.TW_+1SD,0050.TW_-1SD,0050.TW_-2SD,0050.TW_zscore
date,,,,,,,,,,,,,,,,
2021-08-31,137.00,140.35,130.20,140.35,6.604396,4.919981,4.944139,4.869072,4.944139,4.782816,119.440277,148.852515,133.337863,106.991214,95.839696,1.4656
2021-09-30,140.15,143.50,135.20,137.05,6.686813,4.942713,4.966335,4.906755,4.920346,4.791796,120.517572,150.195094,134.540507,107.956224,96.704125,1.1679
2021-10-31,136.30,137.80,130.90,135.80,6.771978,4.914858,4.925803,4.874434,4.911183,4.801074,121.640985,151.595148,135.794636,108.962545,97.605559,1.0004
2021-11-30,136.20,143.55,136.15,138.00,6.854396,4.914124,4.966683,4.913757,4.927254,4.810053,122.738128,152.962464,137.019438,109.945336,98.485914,1.0648
2021-12-31,138.10,146.15,138.05,145.50,6.939560,4.927978,4.984633,4.927616,4.980176,4.819331,123.882240,154.388314,138.296675,110.970199,99.403958,1.4613
2022-01-31,146.00,152.40,141.05,141.55,7.024725,4.983607,5.026509,4.949114,4.952653,4.828610,125.037017,155.827456,139.585817,112.004615,100.330559,1.1270
2022-02-28,142.60,145.05,137.85,138.50,7.101648,4.960044,4.977079,4.926166,4.930870,4.836990,126.089291,157.138853,140.760529,112.947212,101.174911,0.8529
2022-03-31,139.05,140.80,131.00,138.10,7.186813,4.934834,4.947340,4.875197,4.927978,4.846269,127.264641,158.603633,142.072638,114.000058,102.118020,0.7423
2022-04-30,136.60,136.95,125.95,128.75,7.269231,4.917057,4.919616,4.835885,4.857873,4.855248,128.412508,160.034163,143.354066,115.028284,103.039075,0.0238


### Visualizing prediction channels

In [63]:
def timeregchannel(asset, dfohlc, freq='W', startstr='1990-01-01', endstr='2022-12-31', regfield='cl'):
    """Visualize the timeline regression channel and z-score."""
    dfband, regdict = timelinereg(asset, dfohlc, freq, startstr, endstr, regfield)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.3, row_heights=[12.0, 4.0],
                        specs=[[{"type": "candlestick"}], [{"type": "scatter"}]])
    
    fig.add_trace(go.Candlestick(x=dfband.index, open=dfband[f'{asset}_op'], high=dfband[f'{asset}_hi'], 
                                 low=dfband[f'{asset}_lo'], close=dfband[f'{asset}_cl'], name=asset), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_avg'], 
                             mode="lines", name="mean", line_color = '#34acdd'), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_+2SD'], 
                             mode="lines", name="+2SD", line_color = '#e4ad27'), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_+1SD'], 
                             mode="lines", name="+1SD", line_color = '#c69f47'), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_-1SD'], 
                             mode="lines", name="-1SD", line_color = '#9dc647'), 
                  row=1, col=1)
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_-2SD'],
                             mode="lines", name="-2SD", line_color = '#69c647'), 
                  row=1, col=1)
    fig.update_yaxes(type="log", row=1, col=1)
    
    fig.add_trace(go.Scatter(x=dfband.index, y=dfband[f'{asset}_zscore'], name='z-score'), row=2, col=1)    
    
    fig.update_layout(title=f'{asset} regression bands from {startstr} to {endstr}', 
                      title_x=0.5, width=1000, height=800)
    fig.show()

### 0050 Regression channel

In [64]:
timeregchannel(assetlist[0], dfall, 'M', startstr0, endstr0)

### Taiwan Dividend Plus ETF (0056.HK)

In [65]:
timeregchannel(assetlist[1], dfall, 'M', startstr0, endstr0)

### TSMC (2330.TW)

In [66]:
timeregchannel(assetlist[2], dfall, 'M', startstr0, endstr0)

### MediaTek (2454.TW)

In [67]:
timeregchannel(assetlist[3], dfall, 'M', startstr0, endstr0)

### 國泰金 (2882.TW)

In [68]:
timeregchannel(assetlist[4], dfall, 'M', startstr0, endstr0)

### H-share index ETF (2002.TW)

In [69]:
timeregchannel(assetlist[5], dfall, 'M', startstr0, endstr0)

### 大立光 (3008.TW)


In [70]:
timeregchannel(assetlist[6], dfall, 'M', startstr0, endstr0)

### 富邦金 (2881.TW)

In [71]:
timeregchannel(assetlist[7], dfall, 'M', startstr0, endstr0)